## Aramis position jitter
### Display of machine jitter measured along the photonics beamline as a function of pulse ID parity
Refresh data button takes n pulses of live bs data where n is defined in number of shots box

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np
from bsread import source
import seaborn as sns
import pandas as pd
from time import sleep
from IPython.display import display
from IPython.display import clear_output
# import epics as ep
from scipy.optimize import curve_fit
import json

In [2]:
device_name = 'PBPS122'

In [3]:
# channels = ['SAROP11-PBPS117:INTENSITY','SAROP11-PBPS117:XPOS','SAROP11-PBPS117:YPOS']
# channels = ['SLAAR21-LTIM01-EVR0:CALCI','SLAAR21-LTIM01-EVR0:CALCX','SLAAR21-LTIM01-EVR0:CALCY']

if device_name == 'PBPS053':
    channels_proc = ['SARFE10-PBPS053:INTENSITY','SARFE10-PBPS053:XPOS','SARFE10-PBPS053:YPOS']
if device_name == 'PBPS122':
    channels_proc = ['SAROP11-PBPS122:INTENSITY','SAROP11-PBPS122:XPOS','SAROP11-PBPS122:YPOS']   

In [6]:
def get_bs_data(channels, numshots = 100):
    tmp_data = np.zeros([numshots, len(channels)+1])
    with source(channels=channels) as stream:
        for i in range(0,numshots):
            message = stream.receive()
            for ch_num, ch in enumerate(channels):
                tmp_data[i,ch_num] = message.data.data[ch].value
                tmp_data[i,len(channels)] = message.data.pulse_id
                
    data_out = pd.DataFrame(columns=channels)
    for ch_num, ch in enumerate(channels):
        data_out[ch] = tmp_data[:,ch_num]
    data_out['PulseID'] = tmp_data[:,len(channels)]
    # add parity
    vals = []
    for ID in data_out['PulseID']:
        if ID %2 ==0:
            vals.append('Even')
        else:
            vals.append('Odd')
    data_out['Parity'] = vals
    return data_out

def update_plot(plot, channel_x, channel_y, data):
    plot.ax_joint.clear()
    plot.ax_marg_x.clear()
    plot.ax_marg_y.clear()
    
    # centre plot
    sns.kdeplot(
    data=data,
    x=channel_x, y=channel_y,
    hue="Parity",
    ax =plot.ax_joint)
    
    # x margin
    sns.kdeplot(
    data=data,
    x=channel_x,
    hue="Parity",
    legend = False,
    ax =plot.ax_marg_x)
    
    # y margin
    sns.kdeplot(
    data=data,
    y=channel_y,
    hue="Parity",
    legend = False,
    ax =plot.ax_marg_y)

def on_button_clicked(b):
    while continuous.value == True:
        numshots = int(NumShots.value)
        data = get_bs_data(channels_proc, numshots=numshots)

        # Position
        update_plot(plot0,channels_proc[1], channels_proc[2], data)
        # Intensity x
        update_plot(plot1,channels_proc[0], channels_proc[1], data)
        # Intensity y
        update_plot(plot2,channels_proc[0], channels_proc[2], data)
        sleep(10)
    else:
        numshots = int(NumShots.value)
        data = get_bs_data(channels_proc, numshots=numshots)

        # Position
        update_plot(plot0,channels_proc[1], channels_proc[2], data)
        # Intensity x
        update_plot(plot1,channels_proc[0], channels_proc[1], data)
        # Intensity y
        update_plot(plot2,channels_proc[0], channels_proc[2], data)

In [7]:
## Make plot content

In [ ]:
# control widget setup
NumShots = widgets.Text(value = '100', description='Number of shots:')

button = widgets.Button(description="Refresh data")
button.on_click(on_button_clicked)

continuous = widgets.Checkbox(value=False, description='Update loop')
# plot setup
out0 = widgets.Output()
out1 = widgets.Output()
out2 = widgets.Output()

tab = widgets.Tab(children = [out0, out1, out2])
tab.set_title(0, device_name +' Position')
tab.set_title(1, device_name +' Intensity/ horizontal')
tab.set_title(2, device_name +' Intensity/ vertical')

sns.set_theme(style="darkgrid")

with out0:
    plot0 = sns.JointGrid()
    plt.show(plot0)

with out1:
    plot1 = sns.JointGrid()
    plt.show(plot1)
    
with out2:
    plot2 = sns.JointGrid()
    plt.show(plot2)
    
display(button,continuous,NumShots,tab)